In [1]:
from sage.all import *

https://hyperelliptic.org/EFD/g1p/auto-montgom.html

In [14]:
def mynumerator(x):
    if parent(x) == R:
        return x
    return numerator(x)

class fastfrac:
    def __init__(self,top,bot=1):
        if parent(top) == ZZ or parent(top) == R:
            self.top = R(top)
            self.bot = R(bot)
        elif top.__class__ == fastfrac:
            self.top = top.top
            self.bot = top.bot * bot
        else:
            self.top = R(numerator(top))
            self.bot = R(denominator(top)) * bot
    def reduce(self):
        return fastfrac(self.top / self.bot)
    def sreduce(self):
        return fastfrac(I.reduce(self.top),I.reduce(self.bot))
    def iszero(self):
        return self.top in I and not (self.bot in I)
    def isdoublingzero(self):
        return self.top in J and not (self.bot in J)
    def __add__(self,other):
        if parent(other) == ZZ:
            return fastfrac(self.top + self.bot * other,self.bot)
        if other.__class__ == fastfrac:
            return fastfrac(self.top * other.bot + self.bot * other.top,self.bot * other.bot)
        return NotImplemented
    def __sub__(self,other):
        if parent(other) == ZZ:
            return fastfrac(self.top - self.bot * other,self.bot)
        if other.__class__ == fastfrac:
            return fastfrac(self.top * other.bot - self.bot * other.top,self.bot * other.bot)
        return NotImplemented
    def __neg__(self):
        return fastfrac(-self.top,self.bot)
    def __mul__(self,other):
        if parent(other) == ZZ:
            return fastfrac(self.top * other,self.bot)
        if other.__class__ == fastfrac:
            return fastfrac(self.top * other.top,self.bot * other.bot)
        return NotImplemented
    def __rmul__(self,other):
        return self.__mul__(other)
    def __div__(self,other):
        if parent(other) == ZZ:
            return fastfrac(self.top,self.bot * other)
        if other.__class__ == fastfrac:
            return fastfrac(self.top * other.bot,self.bot * other.top)
        return NotImplemented
    __truediv__ = __div__
    def __pow__(self,other):
        if parent(other) == ZZ:
            return fastfrac(self.top ^ other,self.bot ^ other)
        return NotImplemented

def isidentity(x):
    return x.iszero()

def isdoublingidentity(x):
    return x.isdoublingzero()

R.<ua,ub,ux1,uy1,ux2,uy2> = PolynomialRing(QQ,6,order='invlex')
I = R.ideal([
    mynumerator((ub*uy1^2)-(ux1^3+ua*ux1^2+ux1))
    , mynumerator((ub*uy2^2)-(ux2^3+ua*ux2^2+ux2))
])

J = I + R.ideal([0
                    , ux1-ux2
                    , uy1-uy2
                 ])

ua = fastfrac(ua)
ub = fastfrac(ub)
ux1 = fastfrac(ux1)
uy1 = fastfrac(uy1)
ux2 = fastfrac(ux2)
uy2 = fastfrac(uy2)

ux3 = ((ub*(uy2-uy1)^2/(ux2-ux1)^2-ua-ux1-ux2)).reduce()
uy3 = (((fastfrac(2)*ux1+ux2+ua)*(uy2-uy1)/(ux2-ux1)-ub*(uy2-uy1)^3/(ux2-ux1)^3-uy1)).reduce()
ux4 = ((ub*(fastfrac(3)*ux1^2+fastfrac(2)*ua*ux1+fastfrac(1))^2/(fastfrac(2)*ub*uy1)^2-ua-ux1-ux1)).reduce()
uy4 = (((fastfrac(2)*ux1+ux1+ua)*(fastfrac(3)*ux1^2+fastfrac(2)*ua*ux1+fastfrac(1))/(fastfrac(2)*ub*uy1)-ub*(fastfrac(3)*ux1^2+fastfrac(2)*ua*ux1+fastfrac(1))^3/(fastfrac(2)*ub*uy1)^3-uy1)).reduce()
a0 = fastfrac((ub))
a1 = fastfrac((fastfrac(0)))
a2 = fastfrac((ua))
a3 = fastfrac((fastfrac(0)))
a4 = fastfrac((fastfrac(1)))
a6 = fastfrac((fastfrac(0)))
wweierx1 = ((ux1)).reduce().sreduce()
wweiery1 = ((uy1)).reduce().sreduce()
print(isidentity(a0*(wweiery1^2)+a1*(wweierx1*wweiery1)+a3*wweiery1-(((wweierx1+a2)*wweierx1+a4)*wweierx1+a6)))
print(isidentity(ux1-(wweierx1)))
print(isidentity(uy1-(wweiery1)))
wweierx2 = ((ux2)).reduce().sreduce()
wweiery2 = ((uy2)).reduce().sreduce()
wweierx3 = ((ux3)).reduce().sreduce()
wweiery3 = ((uy3)).reduce().sreduce()
wweierx4 = ((ux4)).reduce().sreduce()
wweiery4 = ((uy4)).reduce().sreduce()
slope = ((wweiery2-wweiery1)/(wweierx2-wweierx1)).reduce().sreduce()
print(isidentity(a0*slope^2+a1*slope-a2-wweierx1-wweierx2-wweierx3))
print(isidentity(slope*(wweierx1-wweierx3)-wweiery1-a1*wweierx3-a3-wweiery3))
slope = ((fastfrac(3)*wweierx1^2+fastfrac(2)*a2*wweierx1+a4-a1*wweiery1)/(fastfrac(2)*a0*wweiery1+a1*wweierx1+a3)).reduce().sreduce()
print(isidentity(a0*slope^2+a1*slope-a2-wweierx1-wweierx1-wweierx4))
print(isidentity(slope*(wweierx1-wweierx4)-wweiery1-a1*wweierx4-a3-wweiery4))


True
True
True
True
True
True
True
